In [10]:
import tensorflow as tf

import numpy as np
import os
import time

In [11]:
path_to_file = tf.keras.utils.get_file('data_set.txt', 'https://raw.githubusercontent.com/binhvq/news-corpus/master/sample/demo-full.txt')

In [12]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print(f'Length of text: {len(text)} characters')

Length of text: 153219 characters


In [13]:
# Take a look at the first 250 characters in text
print(text[None:None])

Chây ì nộp phạt nguội.
Hàng chục ngàn phương tiện bị ghi hình vi phạm luật giao thông ở TP.HCM, bị 'bêu tên' nhưng chủ vẫn không chịu nộp phạt.
Trên cổng thông tin điện tử của Công an TP.HCM (CATP), mục thông tin về phương tiện vi phạm hành chính qua hình ảnh (từ ngày 4.1.2017 - 4.1.2018), có ghi nhận biển số xe, lỗi vi phạm, ngày vi phạm của 34.118 phương tiện (ô tô) chưa nộp phạt.
Đây là các phương tiện vi phạm được camera (di động hoặc cố định) của CATP ghi hình phạt nguội .
Điều đáng nói, dù Phòng CSGT đường bộ - đường sắt (PC67), CATP nhiều lần gửi giấy thông báo vi phạm về công an địa phương nhưng chủ hoặc người điều khiển phương tiện vẫn chưa thực hiện quyết định xử phạt hành chính.
Phổ biến nhất là lỗi đỗ không đúng nơi quy định.
Chẳng hạn từ tháng 1 - 7.2017, ô tô BS: 14A-130... 23 lần đỗ không đúng nơi quy định trên đường Hàm Nghi, Q.1; từ tháng 3 - 10.2017, ô tô BS: 30S-087... 34 lần đỗ trên đường Nguyễn Cư Trinh, Q.1; từ tháng 4 - 10.2017, ô tô BS: 86B-0066... 34 lần đỗ trê

In [14]:
# The unique characters in the file
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters')

165 unique characters


In [15]:
example_texts = ['abcdefg', 'xyz']

chars = tf.strings.unicode_split(example_texts, input_encoding='UTF-8')
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [16]:
ids_from_chars = tf.keras.layers.StringLookup(
    vocabulary=list(vocab), mask_token=None)

In [17]:
ids = ids_from_chars(chars)
ids

<tf.RaggedTensor [[55, 56, 57, 58, 59, 60, 61], [77, 78, 0]]>

In [18]:
chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)

In [19]:
chars = chars_from_ids(ids)
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'[UNK]']]>

In [20]:
tf.strings.reduce_join(chars, axis=-1).numpy()

array([b'abcdefg', b'xy[UNK]'], dtype=object)

In [21]:
def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

In [22]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids

<tf.Tensor: shape=(153219,), dtype=int64, numpy=array([32, 62, 86, ...,  4, 13,  1])>

In [23]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [24]:
for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode('utf-8'))

C
h
â
y
 
ì
 
n
ộ
p


In [25]:
seq_length = 100

In [26]:
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

for seq in sequences.take(1):
  print(chars_from_ids(seq))

tf.Tensor(
[b'C' b'h' b'\xc3\xa2' b'y' b' ' b'\xc3\xac' b' ' b'n' b'\xe1\xbb\x99'
 b'p' b' ' b'p' b'h' b'\xe1\xba\xa1' b't' b' ' b'n' b'g' b'u'
 b'\xe1\xbb\x99' b'i' b'.' b'\n' b'H' b'\xc3\xa0' b'n' b'g' b' ' b'c' b'h'
 b'\xe1\xbb\xa5' b'c' b' ' b'n' b'g' b'\xc3\xa0' b'n' b' ' b'p' b'h'
 b'\xc6\xb0' b'\xc6\xa1' b'n' b'g' b' ' b't' b'i' b'\xe1\xbb\x87' b'n'
 b' ' b'b' b'\xe1\xbb\x8b' b' ' b'g' b'h' b'i' b' ' b'h' b'\xc3\xac' b'n'
 b'h' b' ' b'v' b'i' b' ' b'p' b'h' b'\xe1\xba\xa1' b'm' b' ' b'l' b'u'
 b'\xe1\xba\xad' b't' b' ' b'g' b'i' b'a' b'o' b' ' b't' b'h' b'\xc3\xb4'
 b'n' b'g' b' ' b'\xe1\xbb\x9f' b' ' b'T' b'P' b'.' b'H' b'C' b'M' b','
 b' ' b'b' b'\xe1\xbb\x8b' b' ' b"'" b'b'], shape=(101,), dtype=string)


In [27]:
for seq in sequences.take(5):
  print(text_from_ids(seq).numpy())

b"Ch\xc3\xa2y \xc3\xac n\xe1\xbb\x99p ph\xe1\xba\xa1t ngu\xe1\xbb\x99i.\nH\xc3\xa0ng ch\xe1\xbb\xa5c ng\xc3\xa0n ph\xc6\xb0\xc6\xa1ng ti\xe1\xbb\x87n b\xe1\xbb\x8b ghi h\xc3\xacnh vi ph\xe1\xba\xa1m lu\xe1\xba\xadt giao th\xc3\xb4ng \xe1\xbb\x9f TP.HCM, b\xe1\xbb\x8b 'b"
b"\xc3\xaau t\xc3\xaan' nh\xc6\xb0ng ch\xe1\xbb\xa7 v\xe1\xba\xabn kh\xc3\xb4ng ch\xe1\xbb\x8bu n\xe1\xbb\x99p ph\xe1\xba\xa1t.\nTr\xc3\xaan c\xe1\xbb\x95ng th\xc3\xb4ng tin \xc4\x91i\xe1\xbb\x87n t\xe1\xbb\xad c\xe1\xbb\xa7a C\xc3\xb4ng an TP.HCM (CATP), m\xe1\xbb\xa5c"
b' th\xc3\xb4ng tin v\xe1\xbb\x81 ph\xc6\xb0\xc6\xa1ng ti\xe1\xbb\x87n vi ph\xe1\xba\xa1m h\xc3\xa0nh ch\xc3\xadnh qua h\xc3\xacnh \xe1\xba\xa3nh (t\xe1\xbb\xab ng\xc3\xa0y 4.1.2017 - 4.1.2018), c\xc3\xb3 ghi nh\xe1\xba\xadn '
b'bi\xe1\xbb\x83n s\xe1\xbb\x91 xe, l\xe1\xbb\x97i vi ph\xe1\xba\xa1m, ng\xc3\xa0y vi ph\xe1\xba\xa1m c\xe1\xbb\xa7a 34.118 ph\xc6\xb0\xc6\xa1ng ti\xe1\xbb\x87n (\xc3\xb4 t\xc3\xb4) ch\xc6\xb0a n\xe1\xbb\x99p ph\xe1\xba\xa1t.\n\x

In [28]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [29]:
split_input_target(list("Tensorflow"))

(['T', 'e', 'n', 's', 'o', 'r', 'f', 'l', 'o'],
 ['e', 'n', 's', 'o', 'r', 'f', 'l', 'o', 'w'])

In [30]:
dataset = sequences.map(split_input_target)

In [31]:
for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b"Ch\xc3\xa2y \xc3\xac n\xe1\xbb\x99p ph\xe1\xba\xa1t ngu\xe1\xbb\x99i.\nH\xc3\xa0ng ch\xe1\xbb\xa5c ng\xc3\xa0n ph\xc6\xb0\xc6\xa1ng ti\xe1\xbb\x87n b\xe1\xbb\x8b ghi h\xc3\xacnh vi ph\xe1\xba\xa1m lu\xe1\xba\xadt giao th\xc3\xb4ng \xe1\xbb\x9f TP.HCM, b\xe1\xbb\x8b '"
Target: b"h\xc3\xa2y \xc3\xac n\xe1\xbb\x99p ph\xe1\xba\xa1t ngu\xe1\xbb\x99i.\nH\xc3\xa0ng ch\xe1\xbb\xa5c ng\xc3\xa0n ph\xc6\xb0\xc6\xa1ng ti\xe1\xbb\x87n b\xe1\xbb\x8b ghi h\xc3\xacnh vi ph\xe1\xba\xa1m lu\xe1\xba\xadt giao th\xc3\xb4ng \xe1\xbb\x9f TP.HCM, b\xe1\xbb\x8b 'b"


In [32]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<_PrefetchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

In [33]:
# Length of the vocabulary in StringLookup Layer
vocab_size = len(ids_from_chars.get_vocabulary())

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [34]:
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

In [35]:
model = MyModel(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [36]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 166) # (batch_size, sequence_length, vocab_size)


In [37]:
model.summary()

Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  42496     
                                                                 
 gru (GRU)                   multiple                  3938304   
                                                                 
 dense (Dense)               multiple                  170150    
                                                                 
Total params: 4150950 (15.83 MB)
Trainable params: 4150950 (15.83 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [38]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

In [39]:
sampled_indices

array([115,  43, 159, 125, 137,  38,   0,  29, 117,  47,  70,  27, 133,
        65,  76, 120,  37,  80, 108, 162,  29, 144, 122,  92,  29,   6,
        31,  18,  11,  22,  67,  81,  77, 101,  44,   0,  12,  37, 106,
       114,   5, 153, 163,  81,  33, 151, 118,  78,  74, 160,  11, 130,
        55,  25,  89,  75,  17,  26,  24,  44,  51,  63,  56,  75, 127,
       112, 128,   1,  95,   8,  24,  89, 136,  26,  19,  87, 100,  57,
       160,   0,  98,  18,  59,  54,  47, 106,   2,  31, 121, 102, 148,
       156,  63,  74, 153,   4,  77,  42, 164,  50])

In [40]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

Input:
 b'n d\xc3\xa2n t\xe1\xbb\x91i cao, T\xc3\xb2a \xc3\xa1n nh\xc3\xa2n d\xc3\xa2n t\xe1\xbb\x91i cao c\xc3\xb3 hi\xe1\xbb\x87u l\xe1\xbb\xb1c t\xe1\xbb\xab 1/2/2018 quy \xc4\x91\xe1\xbb\x8bnh c\xc6\xa1 quan ti\xe1\xba\xbfn h\xc3\xa0nh t\xe1\xbb\x91 t\xe1\xbb\xa5ng c\xc3\xb3'

Next Char Predictions:
 b'\xe1\xba\xa6O\xe1\xbb\xab\xe1\xba\xb7\xe1\xbb\x8bI[UNK]@\xe1\xba\xa9Sq>\xe1\xbb\x84lw\xe1\xba\xaeH\xc3\x82\xc6\xaf\xe1\xbb\xb1@\xe1\xbb\x97\xe1\xba\xb1\xc3\xad@&B3,7n\xc3\x8ax\xc4\x90P[UNK]-H\xc6\xa0\xe1\xba\xa5%\xe1\xbb\xa3\xe1\xbb\xb3\xc3\x8aD\xe1\xbb\x9f\xe1\xba\xabyu\xe1\xbb\xad,\xe1\xbb\x80a:\xc3\xa9v2;9PXibv\xe1\xba\xbb\xe1\xba\xa3\xe1\xba\xbd\n\xc3\xb4(9\xc3\xa9\xe1\xbb\x89;4\xc3\xa3\xc4\x83c\xe1\xbb\xad[UNK]\xc3\xba3e_S\xc6\xa0 B\xe1\xba\xaf\xc4\x91\xe1\xbb\x9c\xe1\xbb\xa7iu\xe1\xbb\xa3"xN\xe1\xbb\xb7V'


In [41]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [42]:
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", example_batch_mean_loss)

Prediction shape:  (64, 100, 166)  # (batch_size, sequence_length, vocab_size)
Mean loss:         tf.Tensor(5.1121535, shape=(), dtype=float32)


In [43]:
tf.exp(example_batch_mean_loss).numpy()

166.02751

In [44]:
model.compile(optimizer='adam', loss=loss)

In [45]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [46]:
EPOCHS = 20

In [47]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/20
23/23 [==============================] - 15s 626ms/step - loss: 4.8456
Epoch 2/20
23/23 [==============================] - 14s 618ms/step - loss: 3.4840
Epoch 3/20
23/23 [==============================] - 14s 594ms/step - loss: 2.9630
Epoch 4/20
23/23 [==============================] - 13s 548ms/step - loss: 2.5983
Epoch 5/20
23/23 [==============================] - 12s 533ms/step - loss: 2.3672
Epoch 6/20
23/23 [==============================] - 12s 533ms/step - loss: 2.2057
Epoch 7/20
23/23 [==============================] - 13s 569ms/step - loss: 2.0956
Epoch 8/20
23/23 [==============================] - 14s 587ms/step - loss: 2.0128
Epoch 9/20
23/23 [==============================] - 14s 622ms/step - loss: 1.9463
Epoch 10/20
23/23 [==============================] - 14s 621ms/step - loss: 1.8851
Epoch 11/20
23/23 [==============================] - 14s 624ms/step - loss: 1.8230
Epoch 12/20
23/23 [==============================] - 14s 621ms/step - loss: 1.7655
Epoch 13/20
2

In [48]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [49]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

In [50]:
start = time.time()
states = None
next_char = tf.constant(['tôi không'])
result = [next_char]

for n in range(100):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

tôi không đâng 218.2014, đã bỏ hỗ trợ tính vì chưa không khác.
Chủ địch đất phường Theo hưởng cảnh án,, khỏi  

________________________________________________________________________________

Run time: 0.4809577465057373


In [51]:
tf.saved_model.save(one_step_model, 'one_step')
one_step_reloaded = tf.saved_model.load('one_step')

INFO:tensorflow:Assets written to: one_step/assets


INFO:tensorflow:Assets written to: one_step/assets


In [57]:
states = None
next_char = tf.constant(['Linux là họ các.'])
result = [next_char]

for n in range(100):
  next_char, states = one_step_reloaded.generate_one_step(next_char, states=states)
  result.append(next_char)

print(tf.strings.join(result)[0].numpy().decode("utf-8"))

Linux là họ các.
Trường Trúng Văn Đình Thủ tư TKXĐân Trạng; Bá quận tư vờ chính những thông báo đoạn cùng lộ 18.000.
